In [ ]:
import sklearn
import time
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2  # Choose an appropriate scoring function for your data
from sklearn.linear_model import LinearRegression


import numpy as np



In [ ]:
df_train = pd.read_csv('train.csv')
target_col = "price_doc"


In [ ]:
X_train = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
train_columns = X_train.columns.tolist()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the dataset
df_train = pd.read_csv('train.csv')

# Define the target column
target_col = "price_doc"

# Separate features (X) and target (y)
X = df_train.drop(columns=[target_col])
y = df_train[target_col]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Linear Regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Make predictions and evaluate the model
predictions = model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE: ", rmse)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load the test dataset
df_test = pd.read_csv('test.csv')

# Extract the 'row ID' column (assuming it's not a feature)
rowid = df_test['row ID']

# Drop the 'row ID' column from the test dataset for preprocessing
X_test = df_test.drop(columns=['row ID'])

# Apply one-hot encoding (get_dummies) to the test dataset
X_test = pd.get_dummies(X_test, drop_first=True)

# Identify missing columns in the test dataset compared to the training dataset
missing_columns = set(X.columns) - set(X_test.columns)

# Add the missing columns to the test dataset and fill with zeros
for col in missing_columns:
    X_test[col] = 0  # Fill missing columns with zeros

# Ensure the columns are in the same order as in the training dataset
X_test = X_test[X.columns]

# Feature scaling using the same StandardScaler used for training data
X_test_scaled = scaler.transform(X_test)  # Assuming 'scaler' is the trained StandardScaler

# Make predictions on the test data
test_predictions = model.predict(X_test_scaled)

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': rowid, 'price_doc': test_predictions})

# Save the results to a CSV file
result_df.to_csv('test_predictions.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the training data
df_train = pd.read_csv('train.csv')
target_col = "price_doc"

# Prepare the training data
X = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
y = df_train[target_col]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Feature selection with Decision Tree
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

# Get feature importances
importances = dt.feature_importances_

# Select the most important features
important_feature_indices = np.argsort(importances)[::-1][:250] # Select top 20 features
important_features = X_train.columns[important_feature_indices]


In [ ]:

# Filter the training and validation data to include only important features
X_train_selected = X_train[important_features]
X_val_selected = X_val[important_features]

# Scale the selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_val_scaled = scaler.transform(X_val_selected)

# Train the linear regression model on the selected features
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Evaluate the model
predictions = model.predict(X_val_scaled)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
print("RMSE with selected features:", rmse)


In [ ]:

# Prepare the test data
df_test = pd.read_csv('test.csv')
rowid = df_test['row ID']
X_test = pd.get_dummies(df_test.drop(columns=['row ID']), drop_first=True)

# Handle missing columns
missing_columns = set(important_features) - set(X_test.columns)
for col in missing_columns:
    X_test[col] = 0

# Keep only the important features
X_test_selected = X_test[important_features]

# Scale the test data
X_test_scaled = scaler.transform(X_test_selected)

# Make predictions
test_predictions = model.predict(X_test_scaled)

# Create a DataFrame for submission
result_df = pd.DataFrame({'row ID': rowid, 'price_doc': test_predictions})
result_df.to_csv('predictions2.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np

# Load the training data
df_train = pd.read_csv('train.csv')
target_col = "price_doc"

# Prepare the training data
X = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
y = df_train[target_col]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Define the linear regression model
lr = LinearRegression()

# Forward Feature Selection
sfs = SequentialFeatureSelector(lr, n_features_to_select=250, direction='forward', scoring='neg_mean_squared_error')
sfs.fit(X_train, y_train)

# Get the mask of selected features
selected_features = sfs.get_support()


In [ ]:

# Filter the training and validation data to include only selected features
X_train_selected = X_train.loc[:, selected_features]
X_val_selected = X_val.loc[:, selected_features]

# Scale the selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_val_scaled = scaler.transform(X_val_selected)

# Train the linear regression model on the selected features
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Evaluate the model
predictions = model.predict(X_val_scaled)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
print("RMSE with selected features:", rmse)


In [ ]:

# Prepare the test data
df_test = pd.read_csv('test.csv')
rowid = df_test['row ID']
X_test = pd.get_dummies(df_test.drop(columns=['row ID']), drop_first=True)

# Handle missing columns in the test data
missing_columns = set(X_train.columns[selected_features]) - set(X_test.columns)
for col in missing_columns:
    X_test[col] = 0

# Keep only the selected features
X_test_selected = X_test.loc[:, selected_features]

# Scale the test data
X_test_scaled = scaler.transform(X_test_selected)

# Make predictions
test_predictions = model.predict(X_test_scaled)

# Create a DataFrame for submission
result_df = pd.DataFrame({'row ID': rowid, 'price_doc': test_predictions})
result_df.to_csv('predictions_forward_selection.csv', index=False)
